In [18]:
import pandas as pd
import numpy as np
from datetime import datetime

# Load your Excel file (replace with your own path if needed)
try:
	df = pd.read_excel("Employee-Data Prep.xlsx", sheet_name="in")
	print("Shape before cleaning:", df.shape)
	df.head()
except PermissionError:
	print(f"Permission denied: Please close '{file_path}' if it is open in another program and try again.")
except Exception as e:
	print(f"An error occurred: {e}")
df.head()

Shape before cleaning: (155, 17)


employee_id first_name last_name                   email gender department  \
0         1001        Ken    Naidoo  ken.naidoo@example.com   Male         HR   
1         1002        Sam     Smith      sam.smith@mail.com      M       H.R.   
2         1003     Oscar   Mthembu   oscar.mthembu@mail.com   Male        Ops   
3         1004       Sam     Brown        sam.brown@example      M      Sales   
4         1005     Faith    Mthembu  faith.mthembu@mail.com      M       H.R.   

        job_title            hire_date  exit_date  is_active    salary  \
0       Sales Rep           29/05/2017        NaT        1.0  55286.31   
1       Sales Rep  2016-04-27 00:00:00        NaT        1.0  29408.75   
2        Engineer  2015-12-02 00:00:00        NaT        0.0  35393.84   
3  Senior Analyst  2016-01-19 00:00:00        NaT        1.0  39890.20   
4        Engineer  2019-06-18 00:00:00 2022-01-22        NaN  25532.92   

  currency         country         state          city  manager_id  \
0      GBP    south-africa  Western Cape           NaN         NaN   
1      GBP    south-africa           NaN  Johannesburg      1001.0   
2        $              SA           KZN     Cape Town         NaN   
3       Â£             USA           KZN           NaN      1003.0   
4      GBP  United Kingdom           NaN     Polokwane         NaN   

  performance_score  
0               NaN  
1                 C  
2               NaN  
3                 C  
4                 B

In [19]:
# 1. Convert data types
df["hire_date"] = pd.to_datetime(df["hire_date"], errors="coerce", dayfirst=True)
df["exit_date"] = pd.to_datetime(df["exit_date"], errors="coerce", dayfirst=True)
df["salary"] = pd.to_numeric(df["salary"], errors="coerce")
df["manager_id"] = pd.to_numeric(df["manager_id"], errors="coerce")


In [20]:

# 2. Trim & standardize strings
df["first_name"] = df["first_name"].astype(str).str.strip().str.title()
df["last_name"] = df["last_name"].astype(str).str.strip().str.title()
df["email"] = df["email"].astype(str).str.strip().str.lower()


In [21]:

# 3. Standardize department names
department_map = {
    "Hr": "Human Resources", "H.R.": "Human Resources",
    "Ops": "Operations", "Eng": "Engineering",
    "Fin": "Finance", "Sales": "Sales",
    "It" : "Information Technology"
}
df["department"] = df["department"].astype(str).str.strip().str.title()
df["department"] = df["department"].replace(department_map)


In [22]:
# 4. Standardize gender column (M/F → Male/Female)
df["gender"] = df["gender"].astype(str).str.strip().str.upper().replace({
    "M": "Male",
    "F": "Female",
    "MALE": "Male",
    "FEMALE": "Female"
})

In [23]:
# 5. Standardize country names
country_map = {
    "Sa": "South Africa",
    "South-Africa": "South Africa",
    "U.K.": "United Kingdom",
    "Uk": "United Kingdom",
    "USA": "United States",
    "Usa": "United States"
}
df["country"] = df["country"].astype(str).str.strip().str.title()
df["country"] = df["country"].replace(country_map)

In [24]:

# 6. Standardize currency codes
currency_map = {"R": "ZAR", "£": "GBP", "Â£": "GBP", "$": "USD"}
df["currency"] = df["currency"].astype(str).str.strip().str.upper().replace(currency_map)


In [25]:

# 7. Handle missing values
# Exit date → keep NaT if still active
df.loc[(df["is_active"] == 1) & (df["exit_date"].isna()), "exit_date"] = pd.NaT
# Department → fill missing with mode (most common)
if df["department"].isna().sum() > 0:
    df["department"].fillna(df["department"].mode()[0], inplace=True)


In [26]:

# 8. Remove duplicates
df.drop_duplicates(subset="employee_id", keep="first", inplace=True)


In [27]:
# 10. Standardize is_active column
df["is_active"] = df["is_active"].map({1: "Active", 1.0: "Active", 0: "Inactive", 0.0: "Inactive"})


In [28]:

# 9. Basic validations
df["valid_email"] = df["email"].str.match(r".+@.+\..+")
df.loc[df["salary"] < 0, "salary"] = np.nan
df["valid_exit_date"] = (df["exit_date"].isna()) | (df["exit_date"] >= df["hire_date"])

print("Shape after cleaning:", df.shape)


Shape after cleaning: (150, 19)


In [29]:
df.to_csv("Week-3-Employee-Data_CLEANED.csv", index=False)
print("✅ Cleaned file saved as Week-3-Employee-Data_CLEANED.csv")


✅ Cleaned file saved as Week-3-Employee-Data_CLEANED.csv
